# Continuous training pipeline with Kubeflow Pipeline and AI Platform

**Learning Objectives:**
1. Learn how to use Kubeflow Pipeline (KFP) pre-build components (BiqQuery, AI Platform training and predictions)
1. Learn how to use KFP lightweight python components
1. Learn how to build a KFP with these components
1. Learn how to compile, upload, and run a KFP with the command line


In this lab, you will build, deploy, and run a KFP pipeline that orchestrates **BigQuery** and **AI Platform** services to train, tune, and deploy a **scikit-learn** model.


## Understanding the pipeline design


The workflow implemented by the pipeline is defined using a Python based Domain Specific Language (DSL). The pipeline's DSL is in the `covertype_training_pipeline.py` file that we will generate below.

The pipeline's DSL has been designed to avoid hardcoding any environment specific settings like file paths or connection strings. These settings are provided to the pipeline code through a set of environment variables.




## Problem
```bash
grep: pipeline/covertype_training_pipeline.py: No such file or directory
```
This file is under directoy `/lab-02-kfp-pipeline/pipeline`.

## Solution
Copy the file to directory `/lab-02-kfp-pipeline/exercises/pipeline`.

In [2]:
!grep 'BASE_IMAGE =' -A 5 pipeline/covertype_training_pipeline.py

BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')
USE_KFP_SA = os.getenv('USE_KFP_SA')


**NOTE: Because there are no environment variables set, therefore covertype_training_pipeline.py file is missing;  we will create it in the next step.**

The pipeline uses a mix of custom and pre-build components.

- Pre-build components. The pipeline uses the following pre-build components that are included with the KFP distribution:
    - [BigQuery query component](https://github.com/kubeflow/pipelines/tree/0.2.5/components/gcp/bigquery/query)
    - [AI Platform Training component](https://github.com/kubeflow/pipelines/tree/0.2.5/components/gcp/ml_engine/train)
    - [AI Platform Deploy component](https://github.com/kubeflow/pipelines/tree/0.2.5/components/gcp/ml_engine/deploy)
- Custom components. The pipeline uses two custom helper components that encapsulate functionality not available in any of the pre-build components. The components are implemented using the KFP SDK's [Lightweight Python Components](https://www.kubeflow.org/docs/pipelines/sdk/lightweight-python-components/) mechanism. The code for the components is in the `helper_components.py` file:
    - **Retrieve Best Run**. This component retrieves a tuning metric and hyperparameter values for the best run of a AI Platform Training hyperparameter tuning job.
    - **Evaluate Model**. This component evaluates a *sklearn* trained model using a provided metric and a testing dataset.
    

### Exercise

Complete TO DOs the pipeline file below.

<ql-infobox><b>NOTE:</b> If you need help, you may take a look at the complete solution by navigating to **mlops-on-gcp > workshops > kfp-caip-sklearn > lab-02-kfp-pipeline** and opening **lab-02.ipynb**.
</ql-infobox>

In [3]:
%%writefile ./pipeline/covertype_training_pipeline.py
# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""KFP orchestrating BigQuery and Cloud AI Platform services."""

import os

from helper_components import evaluate_model
from helper_components import retrieve_best_run
from jinja2 import Template
import kfp
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

# Defaults and environment settings
BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')
USE_KFP_SA = os.getenv('USE_KFP_SA')

TRAINING_FILE_PATH = 'datasets/training/data.csv'
VALIDATION_FILE_PATH = 'datasets/validation/data.csv'
TESTING_FILE_PATH = 'datasets/testing/data.csv'

# Parameter defaults
SPLITS_DATASET_ID = 'splits'
HYPERTUNE_SETTINGS = """
{
    "hyperparameters":  {
        "goal": "MAXIMIZE",
        "maxTrials": 6,
        "maxParallelTrials": 3,
        "hyperparameterMetricTag": "accuracy",
        "enableTrialEarlyStopping": True,
        "params": [
            {
                "parameterName": "max_iter",
                "type": "DISCRETE",
                "discreteValues": [500, 1000]
            },
            {
                "parameterName": "alpha",
                "type": "DOUBLE",
                "minValue": 0.0001,
                "maxValue": 0.001,
                "scaleType": "UNIT_LINEAR_SCALE"
            }
        ]
    }
}
"""


# Helper functions
def generate_sampling_query(source_table_name, num_lots, lots):
    """Prepares the data sampling query."""

    sampling_query_template = """
         SELECT *
         FROM 
             `{{ source_table }}` AS cover
         WHERE 
         MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
         """
    query = Template(sampling_query_template).render(
        source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

    return query


# Create component factories
#component_store = # TO DO: Complete the command

#bigquery_query_op = # TO DO: Use the pre-build bigquery/query component
#mlengine_train_op = # TO DO: Use the pre-build ml_engine/train
#mlengine_deploy_op = # TO DO: Use the pre-build ml_engine/deploy component
#retrieve_best_run_op = # TO DO: Package the retrieve_best_run function into a lightweight component
#evaluate_model_op = # TO DO: Package the evaluate_model function into a lightweight component

component_store = kfp.components.ComponentStore(
    local_search_paths=None, url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])

bigquery_query_op = component_store.load_component('bigquery/query')
mlengine_train_op = component_store.load_component('ml_engine/train')
mlengine_deploy_op = component_store.load_component('ml_engine/deploy')
retrieve_best_run_op = func_to_container_op(
    retrieve_best_run, base_image=BASE_IMAGE)
evaluate_model_op = func_to_container_op(evaluate_model, base_image=BASE_IMAGE)

@kfp.dsl.pipeline(
    name='Covertype Classifier Training',
    description='The pipeline training and deploying the Covertype classifierpipeline_yaml'
)
def covertype_train(project_id,
                    region,
                    source_table_name,
                    gcs_root,
                    dataset_id,
                    evaluation_metric_name,
                    evaluation_metric_threshold,
                    model_id,
                    version_id,
                    replace_existing_version,
                    hypertune_settings=HYPERTUNE_SETTINGS,
                    dataset_location='US'):
    """Orchestrates training and deployment of an sklearn model."""

    # Create the training split
    query = generate_sampling_query(
        source_table_name=source_table_name, num_lots=10, lots=[1, 2, 3, 4])

    training_file_path = '{}/{}'.format(gcs_root, TRAINING_FILE_PATH)

    create_training_split = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=training_file_path,
        dataset_location=dataset_location)

    # Create the validation split
    query = generate_sampling_query(
        source_table_name=source_table_name, num_lots=10, lots=[8])

    validation_file_path = '{}/{}'.format(gcs_root, VALIDATION_FILE_PATH)

#    create_validation_split = # TODO - use the bigquery_query_op
    create_validation_split = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=validation_file_path,
        dataset_location=dataset_location)

    # Create the testing split
    query = generate_sampling_query(
        source_table_name=source_table_name, num_lots=10, lots=[9])

    testing_file_path = '{}/{}'.format(gcs_root, TESTING_FILE_PATH)

#    create_testing_split = # TO DO: Use the bigquery_query_op
    create_testing_split = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=testing_file_path,
        dataset_location=dataset_location)

    # Tune hyperparameters
    tune_args = [
        '--training_dataset_path',
        create_training_split.outputs['output_gcs_path'],
        '--validation_dataset_path',
        create_validation_split.outputs['output_gcs_path'], '--hptune', 'True'
    ]

    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir/hypertune',
                                kfp.dsl.RUN_ID_PLACEHOLDER)

#    hypertune = # TO DO: Use the mlengine_train_op
    hypertune = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri=TRAINER_IMAGE,
        job_dir=job_dir,
        args=tune_args,
        training_input=hypertune_settings)

    # Retrieve the best trial
    get_best_trial = retrieve_best_run_op(
            project_id, hypertune.outputs['job_id'])

    # Train the model on a combined training and validation datasets
    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir', kfp.dsl.RUN_ID_PLACEHOLDER)

    train_args = [
        '--training_dataset_path',
        create_training_split.outputs['output_gcs_path'],
        '--validation_dataset_path',
        create_validation_split.outputs['output_gcs_path'], '--alpha',
        get_best_trial.outputs['alpha'], '--max_iter',
        get_best_trial.outputs['max_iter'], '--hptune', 'False'
    ]

#    train_model = # TO DO: Use the mlengine_train_op
    train_model = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri=TRAINER_IMAGE,
        job_dir=job_dir,
        args=train_args)

    # Evaluate the model on the testing split
    eval_model = evaluate_model_op(
        dataset_path=str(create_testing_split.outputs['output_gcs_path']),
        model_path=str(train_model.outputs['job_dir']),
        metric_name=evaluation_metric_name)

    # Deploy the model if the primary metric is better than threshold
    with kfp.dsl.Condition(eval_model.outputs['metric_value'] > evaluation_metric_threshold):
        deploy_model = mlengine_deploy_op(
        model_uri=train_model.outputs['job_dir'],
        project_id=project_id,
        model_id=model_id,
        version_id=version_id,
        runtime_version=RUNTIME_VERSION,
        python_version=PYTHON_VERSION,
        replace_existing_version=replace_existing_version)

    # Configure the pipeline to run using the service account defined
    # in the user-gcp-sa k8s secret
    if USE_KFP_SA == 'True':
        kfp.dsl.get_pipeline_conf().add_op_transformer(
              use_gcp_secret('user-gcp-sa'))

Overwriting ./pipeline/covertype_training_pipeline.py


The custom components execute in a container image defined in `base_image/Dockerfile`.

In [4]:
!cat base_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire scikit-learn==0.20.4 pandas==0.24.2 kfp==0.2.5


The training step in the pipeline employes the AI Platform Training component to schedule a  AI Platform Training job in a custom training container. The custom training image is defined in `trainer_image/Dockerfile`.

In [5]:
!cat trainer_image/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]


## Building and deploying the pipeline

Before deploying to AI Platform Pipelines, the pipeline DSL has to be compiled into a pipeline runtime format, also refered to as a pipeline package.  The runtime format is based on [Argo Workflow](https://github.com/argoproj/argo), which is expressed in YAML. 


### Configure environment settings

Update  the below constants  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the GCS bucket created during installation of AI Platform Pipelines. The bucket name will be similar to `qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default`.
- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. Then endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the **SETTINGS** for your instance
2. Use the value of the `host` variable in the **Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD** section of the **SETTINGS** window.

Run gsutil ls without URLs to list all of the Cloud Storage buckets under your default project ID.

In [6]:
!gsutil ls

gs://artifacts.qwiklabs-gcp-00-0a441c097ac4.appspot.com/
gs://qwiklabs-gcp-00-0a441c097ac4-kubeflowpipelines-default/
gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/


**HINT:** 

For **ENDPOINT**, use the value of the `host` variable in the **Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SDK** section of the **SETTINGS** window.

For **ARTIFACT_STORE_URI**, copy the bucket name which starts with the qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default prefix from the previous cell output. Your copied value should look like **'gs://qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default'**


In [7]:
REGION = 'us-central1'
ENDPOINT = '221ed162182180b9-dot-us-central1.pipelines.googleusercontent.com' # TO DO: REPLACE WITH YOUR ENDPOINT
#import kfp
#client = kfp.Client(host='https://221ed162182180b9-dot-us-central1.pipelines.googleusercontent.com')

#ARTIFACT_STORE_URI = 'gs://qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default' # TO DO: REPLACE WITH YOUR ARTIFACT_STORE NAME 
ARTIFACT_STORE_URI = 'gs://qwiklabs-gcp-00-0a441c097ac4-kubeflowpipelines-default' # TO DO
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

### Build the trainer image

In [8]:
IMAGE_NAME='trainer_image'
TAG='latest'
TRAINER_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

#### **Note**: Please ignore any **incompatibility ERROR** that may appear for the packages visions as it will not affect the lab's functionality.

In [9]:
!gcloud builds submit --timeout 15m --tag $TRAINER_IMAGE trainer_image

Creating temporary tarball archive of 2 file(s) totalling 3.4 KiB before compression.
Uploading tarball of [trainer_image] to [gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/source/1671387928.628736-05b8a794d86c454eaee63d34c8cb3caf.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-0a441c097ac4/locations/global/builds/7a4dec99-512e-4670-8a46-a9a542aaa7f2].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/7a4dec99-512e-4670-8a46-a9a542aaa7f2?project=1051368008164 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "7a4dec99-512e-4670-8a46-a9a542aaa7f2"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/source/1671387928.628736-05b8a794d86c454eaee63d34c8cb3caf.tgz#1671387928881028
Copying gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/source/1671387928.628736-05b8a794d86c454eaee63d34c8cb3caf.tgz#1671387928881028...
/ [1 files][  1.7 KiB/  1.7 KiB]                

### Build the base image for custom components

In [10]:
IMAGE_NAME='base_image'
TAG='latest'
BASE_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

## Problem
The following command results in an error message.

```bash
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
visions 0.7.5 requires pandas>=0.25.3, but you have pandas 0.24.2 which is incompatible.
typer 0.7.0 requires click<9.0.0,>=7.1.1, but you have click 7.0 which is incompatible.
statsmodels 0.13.5 requires pandas>=0.25, but you have pandas 0.24.2 which is incompatible.
seaborn 0.12.1 requires pandas>=0.25, but you have pandas 0.24.2 which is incompatible.
phik 0.12.3 requires pandas>=0.25.1, but you have pandas 0.24.2 which is incompatible.
pandas-profiling 3.5.0 requires pandas!=1.4.0,<1.6,>1.1, but you have pandas 0.24.2 which is incompatible.
gym 0.23.1 requires cloudpickle>=1.2.0, but you have cloudpickle 1.1.1 which is incompatible.
google-cloud-bigquery 3.4.1 requires packaging<22.0.0dev,>=14.3, but you have packaging 22.0 which is incompatible.
docker 6.0.1 requires urllib3>=1.26.0, but you have urllib3 1.24.3 which is incompatible.
```

In [19]:
!gcloud builds submit --timeout 15m --tag $BASE_IMAGE base_image

Creating temporary tarball archive of 1 file(s) totalling 122 bytes before compression.
Uploading tarball of [base_image] to [gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/source/1671388187.536451-5ca5c375ca1344c2bf0fb4c563ac8dd0.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-0a441c097ac4/locations/global/builds/4ff4fb07-cf61-403f-8f5f-7f622b346492].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/4ff4fb07-cf61-403f-8f5f-7f622b346492?project=1051368008164 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "4ff4fb07-cf61-403f-8f5f-7f622b346492"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/source/1671388187.536451-5ca5c375ca1344c2bf0fb4c563ac8dd0.tgz#1671388187779129
Copying gs://qwiklabs-gcp-00-0a441c097ac4_cloudbuild/source/1671388187.536451-5ca5c375ca1344c2bf0fb4c563ac8dd0.tgz#1671388187779129...
/ [1 files][  286.0 B/  286.0 B]                 

### Compile the pipeline

You can compile the DSL using an API from the **KFP SDK** or using the **KFP** compiler.

To compile the pipeline DSL using the **KFP** compiler.

#### Set the pipeline's compile time settings

The pipeline can run using a security context of the GKE default node pool's service account or the service account defined in the `user-gcp-sa` secret of the Kubernetes namespace hosting KFP. If you want to use the `user-gcp-sa` service account you change the value of `USE_KFP_SA` to `True`.

Note that the default AI Platform Pipelines configuration does not define the `user-gcp-sa` secret.

In [20]:
USE_KFP_SA = False

COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/'
RUNTIME_VERSION = '1.15'
PYTHON_VERSION = '3.7'

%env USE_KFP_SA={USE_KFP_SA}
%env BASE_IMAGE={BASE_IMAGE}
%env TRAINER_IMAGE={TRAINER_IMAGE}
%env COMPONENT_URL_SEARCH_PREFIX={COMPONENT_URL_SEARCH_PREFIX}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}

env: USE_KFP_SA=False
env: BASE_IMAGE=gcr.io/qwiklabs-gcp-00-0a441c097ac4/base_image:latest
env: TRAINER_IMAGE=gcr.io/qwiklabs-gcp-00-0a441c097ac4/trainer_image:latest
env: COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7


#### Use the CLI compiler to compile the pipeline

### Exercise

Compile the `covertype_training_pipeline.py` with the `dsl-compile` command line:

<ql-infobox><b>NOTE:</b> If you need help, you may take a look at the complete solution by navigating to **mlops-on-gcp > workshops > kfp-caip-sklearn > lab-02-kfp-pipeline** and opening **lab-02.ipynb**.
</ql-infobox>

In [21]:
!dsl-compile --py pipeline/covertype_training_pipeline.py --output covertype_training_pipeline.yaml  # TO DO: Your code goes here

The result is the `covertype_training_pipeline.yaml` file. 

In [22]:
!head covertype_training_pipeline.yaml

"apiVersion": |-
  argoproj.io/v1alpha1
"kind": |-
  Workflow
"metadata":
  "annotations":
    "pipelines.kubeflow.org/pipeline_spec": |-
      {"description": "The pipeline training and deploying the Covertype classifierpipeline_yaml", "inputs": [{"name": "project_id"}, {"name": "region"}, {"name": "source_table_name"}, {"name": "gcs_root"}, {"name": "dataset_id"}, {"name": "evaluation_metric_name"}, {"name": "evaluation_metric_threshold"}, {"name": "model_id"}, {"name": "version_id"}, {"name": "replace_existing_version"}, {"default": "\n{\n    \"hyperparameters\":  {\n        \"goal\": \"MAXIMIZE\",\n        \"maxTrials\": 6,\n        \"maxParallelTrials\": 3,\n        \"hyperparameterMetricTag\": \"accuracy\",\n        \"enableTrialEarlyStopping\": True,\n        \"params\": [\n            {\n                \"parameterName\": \"max_iter\",\n                \"type\": \"DISCRETE\",\n                \"discreteValues\": [500, 1000]\n            },\n            {\n                \"para

#### Set the command fields in the pipeline YAML

In [23]:
!sed -i 's/\"command\": \[\]/\"command\": \[python, -u, -m, kfp_component.launcher\]/g' covertype_training_pipeline.yaml

In [24]:
!cat covertype_training_pipeline.yaml | grep "component.launcher"

      "command": [python, -u, -m, kfp_component.launcher]
      "command": [python, -u, -m, kfp_component.launcher]
      "command": [python, -u, -m, kfp_component.launcher]
      "command": [python, -u, -m, kfp_component.launcher]
      "command": [python, -u, -m, kfp_component.launcher]
      "command": [python, -u, -m, kfp_component.launcher]


You should see 6 lines in the output that were modified by the sed command.

### Deploy the pipeline package

### Exercise

Upload the pipeline to the Kubeflow cluster using the `kfp` command line:

<ql-infobox><b>NOTE:</b> If you need help, you may take a look at the complete solution by navigating to **mlops-on-gcp > workshops > kfp-caip-sklearn > lab-02-kfp-pipeline** and opening **lab-02.ipynb**.
</ql-infobox>

In [25]:
PIPELINE_NAME='covertype_continuous_training'

# TO DO: Your code goes here
!kfp --endpoint $ENDPOINT pipeline upload \
-p $PIPELINE_NAME \
covertype_training_pipeline.yaml

(500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Content-Type': 'text/plain; charset=utf-8', 'Date': 'Sun, 18 Dec 2022 18:33:02 GMT', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-Powered-By': 'Express', 'X-Xss-Protection': '0', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'S=cloud_datalab_tunnel=_FB--d11o2KIv-gQjTbTunGPxeX0onUVcBV91Iku9fM; Path=/; Max-Age=3600'})
HTTP response body: {"error_message":"Error creating pipeline: Create pipeline failed: Already exist error: Failed to create a new pipeline. The name covertype_continuous_training already exist. Please specify a new name.","error_details":"Error creating pipeline: Create pipeline failed: Already exist error: Failed to create a new pipeline. The name covertype_continuous_training already exist. Please specify a new name."}



## Submitting pipeline runs

You can trigger pipeline runs using an API from the KFP SDK or using KFP CLI. To submit the run using KFP CLI, execute the following commands. Notice how the pipeline's parameters are passed to the pipeline run.

### List the pipelines in AI Platform Pipelines

In [26]:
!kfp --endpoint $ENDPOINT pipeline list

+--------------------------------------+------------------------------------------------+---------------------------+
| Pipeline ID                          | Name                                           | Uploaded at               |
+======================================+================================================+===========================+
| 48fb3028-4ea1-4339-a850-599d59de2e94 | covertype_continuous_training                  | 2022-12-18T18:29:42+00:00 |
+--------------------------------------+------------------------------------------------+---------------------------+
| fb5248bc-77a9-4581-949f-53e1ed018465 | [Tutorial] DSL - Control structures            | 2022-12-18T17:47:56+00:00 |
+--------------------------------------+------------------------------------------------+---------------------------+
| 7cc9d880-b739-41be-9e24-86dddb23b44c | [Tutorial] Data passing in python components   | 2022-12-18T17:47:55+00:00 |
+--------------------------------------+----------------

### Submit a run

Find the ID of the `covertype_continuous_training` pipeline you uploaded in the previous step and update the value of `PIPELINE_ID` .




## Problem
Q: What is PIPELINE_ID in this case?
A: 




In [27]:
PIPELINE_ID='0918568d-758c-46cf-9752-e04a4403cd84' # TO DO: REPLACE WITH YOUR PIPELINE ID 

In [28]:
EXPERIMENT_NAME = 'Covertype_Classifier_Training'
RUN_ID = 'Run_001'
SOURCE_TABLE = 'covertype_dataset.covertype'
DATASET_ID = 'splits'
EVALUATION_METRIC = 'accuracy'
EVALUATION_METRIC_THRESHOLD = '0.69'
MODEL_ID = 'covertype_classifier'
VERSION_ID = 'v01'
REPLACE_EXISTING_VERSION = 'True'

GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)

### Exercise

Run the pipeline using the `kfp` command line. Here are some of the variable
you will have to use to pass to the pipeline:

- EXPERIMENT_NAME is set to the experiment used to run the pipeline. You can choose any name you want. If the experiment does not exist it will be created by the command
- RUN_ID is the name of the run. You can use an arbitrary name
- PIPELINE_ID is the id of your pipeline. Use the value retrieved by the   `kfp pipeline list` command
- GCS_STAGING_PATH is the URI to the Cloud Storage location used by the pipeline to store intermediate files. By default, it is set to the `staging` folder in your artifact store.
- REGION is a compute region for AI Platform Training and Prediction.


<ql-infobox><b>NOTE:</b> If you need help, you may take a look at the complete solution by navigating to **mlops-on-gcp > workshops > kfp-caip-sklearn > lab-02-kfp-pipeline** and opening **lab-02.ipynb**.
</ql-infobox>

In [29]:
# TO DO: Your code goes here
!kfp --endpoint $ENDPOINT run submit \
-e $EXPERIMENT_NAME \
-r $RUN_ID \
-p $PIPELINE_ID \
project_id=$PROJECT_ID \
gcs_root=$GCS_STAGING_PATH \
region=$REGION \
source_table_name=$SOURCE_TABLE \
dataset_id=$DATASET_ID \
evaluation_metric_name=$EVALUATION_METRIC \
evaluation_metric_threshold=$EVALUATION_METRIC_THRESHOLD \
model_id=$MODEL_ID \
version_id=$VERSION_ID \
replace_existing_version=$REPLACE_EXISTING_VERSION

Creating experiment Covertype_Classifier_Training.
(404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Sun, 18 Dec 2022 18:33:08 GMT', 'Vary': 'Origin', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'SAMEORIGIN', 'X-Powered-By': 'Express', 'X-Xss-Protection': '0', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'S=cloud_datalab_tunnel=_FB--d11o2KIv-gQjTbTunGPxeX0onUVcBV91Iku9fM; Path=/; Max-Age=3600'})
HTTP response body: {"error":"Validate create run request failed.: Get pipelineId failed.: ResourceNotFoundError: Pipeline 0918568d-758c-46cf-9752-e04a4403cd84 not found.","code":5,"message":"Validate create run request failed.: Get pipelineId failed.: ResourceNotFoundError: Pipeline 0918568d-758c-46cf-9752-e04a4403cd84 not found.","details":[{"@type":"type.googleapis.com/api.Error","error_message":"Pipeline 0918568d-758c-46cf-9752-e04a4403cd84 not found.","error_details":"Validate create run request failed.: Get pipelineId 

### Monitoring the run

You can monitor the run using KFP UI. Follow the instructor who will walk you through the KFP UI and monitoring techniques.

To access the KFP UI in your environment use the following URI:

https://[ENDPOINT]


**NOTE that your pipeline run may fail due to the bug in a BigQuery component that does not handle certain race conditions. If you observe the pipeline failure, re-run the last cell of the notebook to submit another pipeline run or retry the run from the KFP UI**


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>